In [16]:
import pandas as pd
import numpy as np
from pennylane import numpy as np
from sklearn.preprocessing import normalize
from sklearn.preprocessing import StandardScaler

import pennylane as qml
from pennylane.templates.embeddings import AngleEmbedding, AmplitudeEmbedding
from pennylane.optimize import AdamOptimizer
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler


import time
start = time.time()

In [17]:
# Read out CSV and sets/samples creation

df = pd.read_csv('synthetic_data.csv', sep=',')
df = df.astype(float)
train,test = train_test_split(df, test_size=0.30, random_state=2)
train_set = train
test_set = test
train_set = train_set.sample(160)
test_set = test_set.sample(40)
np.random.seed(42)

In [18]:
# Review the information related to the dataframe

df.info()

<bound method DataFrame.info of         X1        X2        X3        X4        X5        X6        X7  \
0      1.0  0.031390  0.031390  0.031390  0.031390  0.031390  0.031390   
1      1.0  5.234565  5.234565  5.234565  5.234565  5.234565  5.234565   
2      1.0  9.354209  9.354209  9.354209  9.354209  9.354209  9.354209   
3      1.0 -8.133407 -8.133407 -8.133407 -8.133407 -8.133407 -8.133407   
4      1.0 -5.471547 -5.471547 -5.471547 -5.471547 -5.471547 -5.471547   
...    ...       ...       ...       ...       ...       ...       ...   
49995  1.0  6.592156  6.592156  6.592156  6.592156  6.592156  6.592156   
49996  1.0 -7.705122 -7.705122 -7.705122 -7.705122 -7.705122 -7.705122   
49997  1.0  5.117075  5.117075  5.117075  5.117075  5.117075  5.117075   
49998  1.0  0.255174  0.255174  0.255174  0.255174  0.255174  0.255174   
49999  1.0  1.638036  1.638036  1.638036  1.638036  1.638036  1.638036   

             X8        X9       X10    y  
0      0.031390  0.031390  0.031390 

In [19]:
# Table of the description of the dataframe related to fixed parameters

df.describe()

,X1,X2,X3,X4,X5,X6,X7,X8,X9,X10,y
count,50000.0,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000
mean,1.0,0.028783,0.028783,0.028783,0.028783,0.028783,0.028783,0.028783,0.028783,0.028783,0.478760
std,0.0,5.761384,5.761384,5.761384,5.761384,5.761384,5.761384,5.761384,5.761384,5.761384,0.499554
min,1.0,-9.999899,-9.999899,-9.999899,-9.999899,-9.999899,-9.999899,-9.999899,-9.999899,-9.999899,0.000000
25%,1.0,-4.926806,-4.926806,-4.926806,-4.926806,-4.926806,-4.926806,-4.926806,-4.926806,-4.926806,0.000000
50%,1.0,0.057610,0.057610,0.057610,0.057610,0.057610,0.057610,0.057610,0.057610,0.057610,0.000000
75%,1.0,4.996073,4.996073,4.996073,4.996073,4.996073,4.996073,4.996073,4.996073,4.996073,1.000000
max,1.0,9.999712,9.999712,9.999712,9.999712,9.999712,9.999712,9.999712,9.999712,9.999712,1.000000


In [20]:
# Separation of labels

x_train = train_set
y_train = train_set[['y']]

x_test = test_set
y_test = test_set[['y']]

In [21]:
# Reduce dimensions using PCA so later you can fit the dimensions with the qubits

n_dim = 2
pca = PCA(n_components=n_dim)
pca.fit(x_train)

x_train = pca.transform(x_train)

pca.fit(x_test)
x_test = pca.transform(x_test)

In [22]:
# Normalize

std_scale = StandardScaler().fit(x_train)
data = std_scale.transform(x_train)

std_scale = StandardScaler().fit(x_test)
x_test = std_scale.transform(x_test)

In [23]:
# Review the balance of the target variable in train

y_train.value_counts(normalize=True)*100

y  
0.0    50.625
1.0    49.375
dtype: float64

In [24]:
# Review the balance of the target variable in test

y_test.value_counts(normalize=True)*100

y  
1.0    57.5
0.0    42.5
dtype: float64

In [25]:
# Angle Encoding

num_qubits = n_dim

dev = qml.device('default.qubit', wires = num_qubits)

@qml.qnode(dev)
def circuit(parameters, data):
    for i in range(num_qubits):
        qml.Hadamard(wires = i)
    
    AngleEmbedding(features = data, wires = range(num_qubits), rotation = 'Y')
    
    qml.StronglyEntanglingLayers(weights = parameters, wires = range(num_qubits))
    
    return qml.expval(qml.PauliZ(0))

In [26]:
num_layers = 5
weights_init = 0.01 * np.random.randn(num_layers, num_qubits, 3, requires_grad=True)
bias_init = np.array(0.0, requires_grad=True)

print(weights_init, bias_init)

[[[ 0.00496714 -0.00138264  0.00647689]
  [ 0.0152303  -0.00234153 -0.00234137]]

 [[ 0.01579213  0.00767435 -0.00469474]
  [ 0.0054256  -0.00463418 -0.0046573 ]]

 [[ 0.00241962 -0.0191328  -0.01724918]
  [-0.00562288 -0.01012831  0.00314247]]

 [[-0.00908024 -0.01412304  0.01465649]
  [-0.00225776  0.00067528 -0.01424748]]

 [[-0.00544383  0.00110923 -0.01150994]
  [ 0.00375698 -0.00600639 -0.00291694]]] 0.0


In [27]:
circuit(weights_init, data[0])

tensor(0.2412114, requires_grad=True)

In [28]:
def variational_classifier(weights, bias, x):
    return circuit(weights, x) + bias

In [29]:
def square_loss(labels, predictions):
    loss = 0
    for l, p in zip(labels, predictions):
        loss = loss + (l - p) ** 2

    loss = loss / len(labels)
    return loss

In [30]:
def accuracy(labels, predictions):

    loss = 0
    for l, p in zip(labels, predictions):
        if abs(l - p) < 1e-5:
            loss = loss + 1
    loss = loss / len(labels)

    return loss

In [31]:
def cost(weights, bias, X, Y):
    predictions = [variational_classifier(weights, bias, x) for x in X]
    return square_loss(Y, predictions)

In [32]:
Y = np.array(y_train.values[:,0] * 2 - np.ones(len(y_train.values[:,0])), requires_grad = False)  # shift label from {0, 1} to {-1, 1}
X = np.array(data, requires_grad=False)

for i in range(5):
    print("X = {}, Y = {: d}".format(list(X[i]), int(Y[i])))

X = [tensor(-0.21657365, requires_grad=False), tensor(-1.49537215, requires_grad=False)], Y =  1
X = [tensor(0.00759177, requires_grad=False), tensor(1.76220068, requires_grad=False)], Y = -1
X = [tensor(0.62203555, requires_grad=False), tensor(0.84566537, requires_grad=False)], Y = -1
X = [tensor(1.7201744, requires_grad=False), tensor(-0.79237387, requires_grad=False)], Y = -1
X = [tensor(-0.89998973, requires_grad=False), tensor(-0.47595428, requires_grad=False)], Y =  1


In [33]:
opt = AdamOptimizer(stepsize=0.1, beta1=0.9, beta2=0.99, eps=1e-08)
batch_size = 10

In [34]:
weights = weights_init
bias = bias_init

wbest = 0
bbest = 0
abest = 0

for it in range(5):

    # weights update by one optimizer step

    batch_index = np.random.randint(0, len(X), (batch_size,))
    X_batch = X[batch_index]
    Y_batch = Y[batch_index]
    weights, bias, _, _ = opt.step(cost, weights, bias, X_batch, Y_batch)

    # Compute the accuracy
    predictions = [np.sign(variational_classifier(weights, bias, x)) for x in X]
    
    if accuracy(Y, predictions) > abest:
        wbest = weights
        bbest = bias
        abest = accuracy(Y, predictions)
        print('New best')

    acc = accuracy(Y, predictions)

    print(
        "Iter: {:5d} | Cost: {:0.7f} | Accuracy: {:0.7f} ".format(
            it + 1, cost(weights, bias, X, Y), acc
        )
    )

New best
Iter:     1 | Cost: 1.0177179 | Accuracy: 0.5437500 
New best
Iter:     2 | Cost: 0.7007890 | Accuracy: 0.6312500 
New best
Iter:     3 | Cost: 0.4357251 | Accuracy: 0.9875000 
New best
Iter:     4 | Cost: 0.2764397 | Accuracy: 1.0000000 
Iter:     5 | Cost: 0.2231992 | Accuracy: 1.0000000 


In [35]:
Yte = np.array(y_test.values[:,0] * 2 - np.ones(len(y_test.values[:,0])), requires_grad = False)
Xte = np.array(normalize(x_test), requires_grad=False)

In [36]:
predictions = [np.sign(variational_classifier(wbest, bbest, x)) for x in Xte]
pred = [np.sign(variational_classifier(wbest, bbest, x)) for x in X]
acc = accuracy(Yte, predictions)

print(f'Cost: {cost(wbest, bbest, Xte, Yte)}, Accuracy: {np.round(acc, 2) * 100}%')

Cost: 0.3223128393661714, Accuracy: 100.0%


In [37]:
pd.DataFrame((predictions, Yte), ('Predictions', 'Test')).T

,Predictions,Test
0,1.0,1.0
1,1.0,1.0
2,1.0,1.0
3,1.0,1.0
4,-1.0,-1.0
5,-1.0,-1.0
6,1.0,1.0
7,1.0,1.0
8,1.0,1.0
9,-1.0,-1.0


In [38]:
# Print the classification report and important metrics

print(metrics.classification_report(predictions,Yte))
print(metrics.precision_score(predictions,Yte))
print(metrics.recall_score(predictions,Yte))
print(metrics.f1_score(predictions,Yte))
print(metrics.balanced_accuracy_score(predictions,Yte))

              precision    recall  f1-score   support

        -1.0       1.00      1.00      1.00        17
         1.0       1.00      1.00      1.00        23

    accuracy                           1.00        40
   macro avg       1.00      1.00      1.00        40
weighted avg       1.00      1.00      1.00        40

1.0
1.0
1.0
1.0
